In [11]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## 1. TF (Term Frequency) et TF-IDF

### 2.1 Concept de TF (Term Frequency)

**Définition** : La fréquence d'un terme dans un document.

**Formule** : TF(t, d) = (Nombre de fois où le terme t apparaît dans le document d) / (Nombre total de termes dans d)

### Exemple simple

In [12]:
# Corpus de documents
documents = [
    "le chat mange la souris",
    "le chien mange les croquettes",
    "la souris est petite"
]

# Calcul manuel du TF
doc1 = "le chat mange la souris"
mots = doc1.split()
print(f"Document : {doc1}")
print(f"Nombre total de mots : {len(mots)}")

# Compter chaque mot
from collections import Counter
compteur = Counter(mots)
print(f"\nFréquences brutes : {compteur}")

# TF normalisé
tf_doc1 = {mot: count/len(mots) for mot, count in compteur.items()}
print(f"\nTF normalisé : {tf_doc1}")

Document : le chat mange la souris
Nombre total de mots : 5

Fréquences brutes : Counter({'le': 1, 'chat': 1, 'mange': 1, 'la': 1, 'souris': 1})

TF normalisé : {'le': 0.2, 'chat': 0.2, 'mange': 0.2, 'la': 0.2, 'souris': 0.2}


### 1.2 TF-IDF (Term Frequency - Inverse Document Frequency)

**Problème avec TF** : Les mots fréquents comme "le", "la" ont des scores élevés mais peu d'importance.

**Solution** : Pénaliser les mots qui apparaissent dans beaucoup de documents.

**Formule** : TF-IDF(t, d) = TF(t, d) × IDF(t)

Où IDF(t) = log(Nombre total de documents / Nombre de documents contenant t)
### Exemple avec scikit-learn

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Nos documents
documents = [
    "le chat mange la souris",
    "le chien mange les croquettes",
    "la souris est petite"
]

# Créer le vectoriseur TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Afficher les mots du vocabulaire
print("Vocabulaire :", vectorizer.get_feature_names_out())
print("\nMatrice TF-IDF :")
print(tfidf_matrix.toarray())

# Créer un DataFrame pour mieux visualiser
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=['Doc1', 'Doc2', 'Doc3']
)
print("\n", df_tfidf)

Vocabulaire : ['chat' 'chien' 'croquettes' 'est' 'la' 'le' 'les' 'mange' 'petite'
 'souris']

Matrice TF-IDF :
[[0.54935123 0.         0.         0.         0.41779577 0.41779577
  0.         0.41779577 0.         0.41779577]
 [0.         0.49047908 0.49047908 0.         0.         0.37302199
  0.49047908 0.37302199 0.         0.        ]
 [0.         0.         0.         0.5628291  0.42804604 0.
  0.         0.         0.5628291  0.42804604]]

           chat     chien  croquettes       est        la        le       les  \
Doc1  0.549351  0.000000    0.000000  0.000000  0.417796  0.417796  0.000000   
Doc2  0.000000  0.490479    0.490479  0.000000  0.000000  0.373022  0.490479   
Doc3  0.000000  0.000000    0.000000  0.562829  0.428046  0.000000  0.000000   

         mange    petite    souris  
Doc1  0.417796  0.000000  0.417796  
Doc2  0.373022  0.000000  0.000000  
Doc3  0.000000  0.562829  0.428046  


## 2. Naive Bayes

### 2.1 Concept

**Naive Bayes** est un algorithme de classification basé sur le théorème de Bayes.

**Théorème de Bayes** : P(Classe|Document) = P(Document|Classe) × P(Classe) / P(Document)

**"Naive"** : On suppose que les mots sont indépendants (hypothèse naïve mais efficace en pratique).

### 2.2 Exemple simple : Classification de sentiments

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Dataset d'exemple : avis de films
textes = [
    "ce film est excellent j'ai adoré",
    "superbe film très bon",
    "film magnifique bravo",
    "film horrible très mauvais",
    "nul je déteste ce film",
    "catastrophe film affreux"
]

# Labels : 1 = positif, 0 = négatif
labels = [1, 1, 1, 0, 0, 0]

# Étape 1 : Vectorisation (convertir texte en nombres)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(textes)

print("Vocabulaire :", vectorizer.get_feature_names_out())
print("\nMatrice de comptage :")
print(X.toarray())

# Étape 2 : Entraîner le modèle
model = MultinomialNB()
model.fit(X, labels)

# Étape 3 : Prédire de nouveaux textes
nouveaux_textes = [
    "film excellent",
    "film horrible"
]

X_nouveaux = vectorizer.transform(nouveaux_textes)
predictions = model.predict(X_nouveaux)

for texte, pred in zip(nouveaux_textes, predictions):
    sentiment = "POSITIF" if pred == 1 else "NÉGATIF"
    print(f"'{texte}' → {sentiment}")

Vocabulaire : ['adoré' 'affreux' 'ai' 'bon' 'bravo' 'catastrophe' 'ce' 'déteste' 'est'
 'excellent' 'film' 'horrible' 'je' 'magnifique' 'mauvais' 'nul' 'superbe'
 'très']

Matrice de comptage :
[[1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0]
 [0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]]
'film excellent' → POSITIF
'film horrible' → NÉGATIF


### 2.3 Exemple plus réaliste : Classification de catégories

In [15]:
# Dataset : classification d'articles
articles = [
    "le match de football était passionnant",
    "le championnat de tennis commence demain",
    "nouvelle victoire pour l'équipe de basketball",
    "le nouveau smartphone est révolutionnaire",
    "l'ordinateur portable est très performant",
    "la tablette tactile a un bel écran",
    "le gouvernement annonce une nouvelle loi",
    "le président rencontre les députés",
    "débat politique sur les réformes"
]

categories = [
    "sport", "sport", "sport",
    "tech", "tech", "tech",
    "politique", "politique", "politique"
]

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(
    articles, categories, test_size=0.33, random_state=42
)

In [16]:
# Vectorisation
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Entraînement
classifier = MultinomialNB()
classifier.fit(X_train_vec, y_train)

# Évaluation
predictions = classifier.predict(X_test_vec)
print(f"Précision : {accuracy_score(y_test, predictions):.2%}")
print("\nRapport de classification :")
print(classification_report(y_test, predictions))

# Prédire de nouveaux articles
nouveaux_articles = [
    "le joueur marque un but incroyable",
    "lancement du nouveau téléphone portable"
]
X_nouveaux = vectorizer.transform(nouveaux_articles)
predictions = classifier.predict(X_nouveaux)

for article, categorie in zip(nouveaux_articles, predictions):
    print(f"'{article}' → Catégorie : {categorie.upper()}")

Précision : 66.67%

Rapport de classification :
              precision    recall  f1-score   support

   politique       0.50      1.00      0.67         1
       sport       1.00      1.00      1.00         1
        tech       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3

'le joueur marque un but incroyable' → Catégorie : TECH
'lancement du nouveau téléphone portable' → Catégorie : TECH


C:\Users\nessr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nessr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nessr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [17]:
## 3. Exemple complet : Détecteur de spam

In [18]:
# Dataset de SMS (spam ou non)
messages = [
    "Félicitations! Vous avez gagné 1000€! Cliquez ici",
    "Promotion exceptionnelle! Offre limitée aujourd'hui seulement",
    "URGENT: Votre compte sera fermé. Cliquez maintenant",
    "Salut, on se voit ce soir pour le cinéma?",
    "N'oublie pas la réunion de demain à 14h",
    "Merci pour ton message, je te rappelle plus tard",
    "100% GRATUIT! Téléchargez maintenant",
    "Voulez-vous dîner ensemble samedi?",
    "Rendez-vous chez le médecin confirmé pour lundi",
    "GAGNEZ de l'argent facilement en travaillant de chez vous"
]
# 1 = spam, 0 = non spam
etiquettes = [1, 1, 1, 0, 0, 0, 1, 0, 0, 1]


In [19]:
# Pipeline complète
print("=== DÉTECTEUR DE SPAM ===\n")

# 1. Vectorisation avec TF-IDF
vectorizer = TfidfVectorizer(lowercase=True, max_features=50)
X = vectorizer.fit_transform(messages)

# 2. Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, etiquettes, test_size=0.3, random_state=42
)

# 3. Entraînement du modèle
model = MultinomialNB()
model.fit(X_train, y_train)

# 4. Évaluation
y_pred = model.predict(X_test)
print(f"Précision sur le test : {accuracy_score(y_test, y_pred):.2%}\n")

# 5. Prédictions sur de nouveaux messages
nouveaux_messages = [
    "Réunion annulée, on reporte à demain",
    "GAGNEZ 5000€ SANS EFFORT cliquez ICI",
    "Tu viens au restaurant ce midi?"
]

print("=== PRÉDICTIONS ===")
X_nouveaux = vectorizer.transform(nouveaux_messages)
predictions = model.predict(X_nouveaux)
probas = model.predict_proba(X_nouveaux)

for msg, pred, proba in zip(nouveaux_messages, predictions, probas):
    type_msg = "SPAM" if pred == 1 else "OK"
    confiance = proba[pred] * 100
    print(f"{type_msg} ({confiance:.1f}%) : '{msg}'")

=== DÉTECTEUR DE SPAM ===

Précision sur le test : 33.33%

=== PRÉDICTIONS ===
OK (58.4%) : 'Réunion annulée, on reporte à demain'
SPAM (71.2%) : 'GAGNEZ 5000€ SANS EFFORT cliquez ICI'
OK (52.0%) : 'Tu viens au restaurant ce midi?'


# Les N-grammes

## Définition

Un **n-gramme** est une séquence de **n** mots consécutifs dans un texte.

- **1-gramme** (unigramme) = 1 mot
- **2-grammes** (bigramme) = 2 mots consécutifs
- **3-grammes** (trigramme) = 3 mots consécutifs
- etc.

## Exemple concret

Prenons la phrase : **"le chat mange la souris"**

### Unigrammes (1-gramme)
```
["le", "chat", "mange", "la", "souris"]
```

### Bigrammes (2-grammes)
```
["le chat", "chat mange", "mange la", "la souris"]
```

### Trigrammes (3-grammes)
```
["le chat mange", "chat mange la", "mange la souris"]
```

Les n-grammes capturent le **contexte** et les **expressions** :

**Sans n-grammes** : "pas" et "bon" sont séparés
**Avec bigrammes** : "pas bon" est reconnu comme une expression négative

## Exemple

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

documents = [
    "le film est très bon",
    "le film est pas bon",
    "ce film est excellent"
]

# Unigrammes seulement (par défaut)
vectorizer_uni = CountVectorizer()
X_uni = vectorizer_uni.fit_transform(documents)

print("=== UNIGRAMMES ===")
print("Vocabulaire :", vectorizer_uni.get_feature_names_out())
print()

# Bigrammes seulement
vectorizer_bi = CountVectorizer(ngram_range=(2, 2))
X_bi = vectorizer_bi.fit_transform(documents)

print("=== BIGRAMMES ===")
print("Vocabulaire :", vectorizer_bi.get_feature_names_out())
print()

# Unigrammes + Bigrammes
vectorizer_mix = CountVectorizer(ngram_range=(1, 2))
X_mix = vectorizer_mix.fit_transform(documents)

print("=== UNIGRAMMES + BIGRAMMES ===")
print("Vocabulaire :", vectorizer_mix.get_feature_names_out())

=== UNIGRAMMES ===
Vocabulaire : ['bon' 'ce' 'est' 'excellent' 'film' 'le' 'pas' 'très']

=== BIGRAMMES ===
Vocabulaire : ['ce film' 'est excellent' 'est pas' 'est très' 'film est' 'le film'
 'pas bon' 'très bon']

=== UNIGRAMMES + BIGRAMMES ===
Vocabulaire : ['bon' 'ce' 'ce film' 'est' 'est excellent' 'est pas' 'est très'
 'excellent' 'film' 'film est' 'le' 'le film' 'pas' 'pas bon' 'très'
 'très bon']


In [21]:
### Impact sur la classification

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Avis de restaurant
avis = [
    "la nourriture est bonne mais le service est mauvais",
    "excellent restaurant le service est parfait",
    "nourriture pas bonne et service lent",
    "très bon restaurant service rapide",
    "décevant la nourriture est pas terrible",
    "parfait tout était bon le service aussi"
]

sentiments = [0, 1, 0, 1, 0, 1]  # 0=négatif, 1=positif

In [23]:
print("=== COMPARAISON UNIGRAMMES vs BIGRAMMES ===\n")

# Test 1 : Unigrammes seulement
vectorizer1 = TfidfVectorizer(ngram_range=(1, 1))
X1 = vectorizer1.fit_transform(avis)
model1 = MultinomialNB()
model1.fit(X1, sentiments)

test_avis = ["nourriture pas bonne", "service très bon"]
X_test1 = vectorizer1.transform(test_avis)
pred1 = model1.predict(X_test1)

print("AVEC UNIGRAMMES SEULEMENT :")
for avis_test, pred in zip(test_avis, pred1):
    print(f"  '{avis_test}' → {'POSITIF' if pred == 1 else 'NÉGATIF'}")

=== COMPARAISON UNIGRAMMES vs BIGRAMMES ===

AVEC UNIGRAMMES SEULEMENT :
  'nourriture pas bonne' → NÉGATIF
  'service très bon' → POSITIF


In [24]:
# Test 2 : Unigrammes + Bigrammes
vectorizer2 = TfidfVectorizer(ngram_range=(1, 2))
X2 = vectorizer2.fit_transform(avis)
model2 = MultinomialNB()
model2.fit(X2, sentiments)

X_test2 = vectorizer2.transform(test_avis)
pred2 = model2.predict(X_test2)

print("\nAVEC UNIGRAMMES + BIGRAMMES :")
for avis_test, pred in zip(test_avis, pred2):
    print(f"  '{avis_test}' → {'POSITIF' if pred == 1 else 'NÉGATIF'}")


AVEC UNIGRAMMES + BIGRAMMES :
  'nourriture pas bonne' → NÉGATIF
  'service très bon' → POSITIF


In [25]:
# Montrer les bigrammes détectés
print("\n=== BIGRAMMES IMPORTANTS DÉTECTÉS ===")
bigrammes = [mot for mot in vectorizer2.get_feature_names_out() if ' ' in mot]
print(f"Exemples : {bigrammes[:10]}")


=== BIGRAMMES IMPORTANTS DÉTECTÉS ===
Exemples : ['bon le', 'bon restaurant', 'bonne et', 'bonne mais', 'décevant la', 'est bonne', 'est mauvais', 'est parfait', 'est pas', 'et service']


In [26]:
### Exemple 4 : Cas d'usage réel - Détection d'expressions

In [27]:
# Expressions idiomatiques et négations
textes = [
    "ce n'est pas mal",          # Négation importante
    "pas du tout satisfait",      # Double négation
    "machine à laver",            # Expression figée
    "pomme de terre",             # Expression figée
    "tout à fait d'accord",       # Expression positive
    "pas d'accord du tout"        # Expression négative
]

In [28]:
# Avec unigrammes : perd le sens
vectorizer_uni = CountVectorizer(ngram_range=(1, 1))
X_uni = vectorizer_uni.fit_transform(textes)

print("UNIGRAMMES (perd le contexte) :")
print(vectorizer_uni.get_feature_names_out())
print()

UNIGRAMMES (perd le contexte) :
['accord' 'ce' 'de' 'du' 'est' 'fait' 'laver' 'machine' 'mal' 'pas'
 'pomme' 'satisfait' 'terre' 'tout']



In [29]:
# Avec bigrammes et trigrammes : capture le sens
vectorizer_multi = CountVectorizer(ngram_range=(1, 3))
X_multi = vectorizer_multi.fit_transform(textes)

print("AVEC N-GRAMMES (capture les expressions) :")
ngrams_detectes = [mot for mot in vectorizer_multi.get_feature_names_out() if ' ' in mot]
print(ngrams_detectes)

AVEC N-GRAMMES (capture les expressions) :
['accord du', 'accord du tout', 'ce est', 'ce est pas', 'de terre', 'du tout', 'du tout satisfait', 'est pas', 'est pas mal', 'fait accord', 'machine laver', 'pas accord', 'pas accord du', 'pas du', 'pas du tout', 'pas mal', 'pomme de', 'pomme de terre', 'tout fait', 'tout fait accord', 'tout satisfait']


## Quand utiliser les n-grammes ?

### Utiliser les n-grammes quand :

1. **Négations** : "pas bon", "n'est pas", "ne... pas"
2. **Expressions figées** : "machine à laver", "arc en ciel"
3. **Noms composés** : "New York", "intelligence artificielle"
4. **Expressions de sentiment** : "très bien", "pas mal", "tout à fait"
5. **Contexte important** : "service client", "retour produit"

### Inconvénients des n-grammes :

1. **Augmente la dimensionnalité** : beaucoup plus de features
2. **Risque de surapprentissage** : avec peu de données
3. **Plus lent** : calculs plus complexes
4. **Sparsité** : matrices très creuses

## Recommandations pratiques

```python
# Configuration typique recommandée
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),      # Unigrammes + Bigrammes
    max_features=5000,       # Limiter le nombre de features
    min_df=2                 # Ignorer les mots trop rares
)
```